# Extremum Seeking

Two-dimensional Extremum Seeking Algorithm that minimizes or maximizes a single local objective function.

The ES searches over 2 dimensions, with probes that have the same frequency, but are offset by $\pi/2$. The probes may have different amplitudes, and each channel may have a different integrator gain.

<!-- $\left( \theta + a \sin \omega t \right)^{2}$

$\theta^{2} + 2 \theta a \sin \omega t + a^{2} \sin^{2} \omega t $

$\theta^{2} + 2 \theta a \sin \omega t + + a^{2} \frac{1}{2} \left( 1 - \cos 2 \omega t \right)$ -->

# ES

Extremum Seeking (ES) is a model-free optimization and control algorithm, that can be used in real-time. ES operates to minimize of maximize an objective function value, which is either fed into the ES algorithm, or calculated from measurements fed into th ES algorithm. ES approximates gradient descent, in that it estimates the gradient of the objective function with respect to its setpoint (system input), and moves its setpoint accordingly.

<!-- ## ES Operation
ES operates as follows:

The system input is formed by adding a sinusoidal perturbation $a_{c} \cos \left( \omega t \right)$ to the ES setpoint $\hat{\theta}_{c}$, such that $u = \hat{\theta}_{c} + a \sin \left( \omega t \right)$. The setpoint is the estimate of the (local) optimizer.

$u = \hat{\theta}_{c} + a_{c} \cos \left( \omega t \right)$

$v = \hat{\theta}_{s} + a_{s} \sin \left( \omega t \right)$

The system inputs $u, v$ passes through the unknown system, resulting in system output (measurements) $bf{y} \left( u, v \right)$.

Either a central entity computes the objective function value $\Psi = \Psi \left( \textbf{y} \right)$, or the ES algorithm may take in the measurements $\textbf{y}$, and compute $\Psi$ itself. -->


## <center> ES Operation </center>

ES approximates gradient descent by steering its optimizer estimate, $\hat{\theta}$, with an estimate of the gradient of an objective function to $\hat{\theta}$.

ES operates as follows:

The system input is formed by adding a sinusoidal perturbation $a \sin \left( \omega t \right)$ to the ES setpoint $\hat{\theta}$, such that $\theta = \hat{\theta} + a \sin \left( \omega t \right)$. The setpoint is the estimate of the (local) optimizer.

The system input $\theta$ passes through the unknown system, resulting in system output (measurements) $\textbf{y} \left( \textbf{u} \right)$.

Either a central entity computes the objective function value $\Psi = \Psi \left( \bf{y} \right)$, or the ES algorithm may take in the measurements $\bf{y}$, and compute $\Psi$ itself.

A high-pass filter removes low-frequency content from $\Psi$, giving $\rho$

This value is demodulated by multiplying by $\sin \left( \omega t \right)$, giving $\sigma$.

A low-pass filter removes high-frequency content from this value, giving the gradient estimate, $\hat{\xi}$.

The gradient estimate is integrated back into the setpoint (optimizer estimate), updating $\hat{\theta}$.





## <center> Extremum Seeking Parameters </center>

This section describes the parameters of an instance of 2D ES with sinusoidal perturbation.

### Parameters for Simple ES

#### Perturbation Frequency: $f$ [Hz]
The perturbation frequency $f$ determines how fast the sinusoidal perturbation cycles. This is converted into an angular frequency $\omega = 2 \pi f$.

#### Perturbation Amplitude: $a$
The perturbation amplitude $a$ determines how far into the search space the perturbation reaches. Larger values tend to provide faster convergence to an optimizer, as the ES can assess more of the search space. Smaller values may provide slower convergence to an optimizer, but the control value will remain closer to the optimizer after convergence.

#### Integrator Gain: $b$
The integerator gain $b$ determines how far the ES algorithm will move in the direction of the gradient estimate. This value essentially scales the gradient estimate. This value can be though of as similar to the step size in gradient descent. Larger values generally provide faster convergence to an optimizer, but may lead to instability.

### Parameters for Advanced ES

#### High-pass filter frequency: $\omega_h$
The high-pass filter frequency determines what frequency content passes through the high-pass filter. This value is typically set to $\omega_{h} = 0.1 \omega$, and SimpleES classes default to this value.

#### Low-pass filter frequency: $\omega_l$
The low-pass filter frequency determines what frequency content passes through the low-pass filter. The low-pass filter attenuates content in the demodulated signal, and typically is used to attenuate content such as that due to the perturbation. This value is typically set to $\omega_{l} = 0.1 \omega$, and SimpleES classes default to this value.


## <center> Inside the 2D ES algorithm </center>

Here, we define $s$ as the Laplace variable, $k$ as the index for discretized equations, and $T$ as the discretized timestep.

We assume that both channels share the same high-pass filter and low-pass filter parameters.

We denote the two channels by subscripts $c$ and $s$, corresponding to the channels perturbed by $\cos \left( \omega t \right)$, and $\sin \left( \omega t \right)$, respectively.

#### Objective function: $\Psi$
The ES algorithm received the objective function value $\Psi$

#### $\rho$: Changes in the objectuve function due to the perturbation
The objective function value passes through a high-pass filter to remove low frequency content, such as changes in the objective function value due to changes in the setpoint. Changes in the objective function value due to the sinusoidal perturbation pass through the filter. The outputs of the high-pass filters are $\rho_{c}$ and $\rho_{s}$:

<center>
Continuous representation:
$\rho_{c} = \displaystyle \frac{s}{s + \omega_{h}} \Psi$
$\quad$
$\rho_{s} = \displaystyle \frac{s}{s + \omega_{h}} \Psi$
</center>

<center>
Discretized representation:
$\rho_{c,k} = \left( 1 - T \omega_{h} \right) \rho_{c,k-1} + \Psi_{k} - \Psi_{k-1}$
$\quad$
$\rho_{s,k} = \left( 1 - T \omega_{h} \right) \rho_{s,k-1} + \Psi_{k} - \Psi_{k-1}$
</center>

#### $\sigma$: Demodulated value
The high-pass filtered objective function values $\rho_{c}$ and $\rho_{s}$ are demodulated by multiplying it by the sinusoidal perturbations and dividing by the perturbation ampltiude, giving $\sigma_{c}$ and $\sigma_{s}$:

<center>
Continuous representation:
$\sigma_{c} = \displaystyle \frac{2}{a_{c}} \cos \left( \omega t \right) \rho_{c}$
$\quad$
$\sigma_{s} = \displaystyle \frac{2}{a_{s}} \sin \left( \omega t \right) \rho_{s}$
</center>

<center>
Discretized representation:
$\sigma_{c,k} = \displaystyle \frac{2}{a_{c}} \cos \left( \omega t \right) \rho_{c,k}$
$\quad$
$\sigma_{s,k} = \displaystyle \frac{2}{a_{s}} \sin \left( \omega t \right) \rho_{s,k}$
</center>

#### $\hat{\xi}$: Gradient Estimate
The demodulated value passes through a low-pass filter to remove sinsoidal and other high frequency content, giving an estimate of the gradient of the objective function $\Psi$ with respect to setpoints $\hat{\theta}_{c}$, and $\hat{\theta}_{s}$, $\hat{\xi}_{c}$, and $\hat{\xi}_{s}$, respectively:

<center>
Continuous representation:
$\hat{\xi}_{c} = \displaystyle \frac{\omega_{l}}{s + \omega_{l}} \sigma_{c}$
$\quad$
$\hat{\xi}_{s} = \displaystyle \frac{\omega_{l}}{s + \omega_{l}} \sigma_{s}$
</center>

<center>
Discrete representation:
$\hat{\xi}_{c,k} = \left( 1 - T \omega_{l} \right) \hat{\xi}_{c,k-1} + T \omega_{l} \sigma_{c,k-1}$
$\quad$
$\hat{\xi}_{s,k} = \left( 1 - T \omega_{l} \right) \hat{\xi}_{s,k-1} + T \omega_{l} \sigma_{s,k-1}$
</center>

#### $\hat{\theta}$: Setpoint
The ES algotrithm then integrates its gradient estimate, scaled by a gain $k$, to update its setpoint $\hat{\theta}$. Positive values of $k$ are used to maximize $\Psi$, and negative values of $k$ are used to minimize $\Psi$ :

<center>
Continuous representation:
$\hat{\theta}_{c} = \displaystyle \pm \frac{b_{c}}{s} \hat{\xi}_{c}$
$\quad$
$\hat{\theta}_{s} = \displaystyle \pm \frac{b_{s}}{s} \hat{\xi}_{s}$
</center>

<center>
Discrete representation:
$\hat{\theta}_{c,k} = \hat{\theta}_{c,k-1} + T b_{c} \hat{\xi}_{c,k-1}$
$\quad$
$\hat{\theta}_{s,k} = \hat{\theta}_{s,k-1} + T b_{s} \hat{\xi}_{s,k-1}$
</center>

#### $\theta$: Control
The ES algotrithm adds the perturbation to its setpointto update its setpoint $\hat{\theta}$, giving the control value, $\theta$:

<center>
Continuous representation:
$\theta_{c} = \hat{\theta}_{c} + a_{c} \cos \left( \omega t \right)$
$\quad$
$\theta_{c} = \hat{\theta}_{c} + a_{c} \cos \left( \omega t \right)$
</center>

<center>
Discretized representation:
$\theta_{c,k} = \hat{\theta}_{c,k} + a_{c} \sin \left( \omega k T \right)$
$\quad$
$\theta_{c,k} = \hat{\theta}_{c,k} + a_{c} \sin \left( \omega k T \right)$
</center>

In [ ]:
# Imports

import numpy as np
import matplotlib.pyplot as plt

from lib.ExtremumSeekingSimple1D import ExtremumSeekingSimple1D
from lib.ExtremumSeekingSimple2D import ExtremumSeekingSimple2D
from lib.ExtremumSeekingSimpleND import ExtremumSeekingSimpleND

from lib.System_Module import PassThroughSystem

from lib.Objective_Function_Module import ObjectiveFunction

from lib.Simulation_Module import Simulation

from lib.Plotting_Module import plot_es_results



### Example 1: Simulation of 2d ES with simple passthrough system

In this example, a single 2D ESC minimizes the value of an objective function, by estimating the optimal input to an unknown system.

System: $\textbf{y} ( t ) = \textbf{u} ( t )$

Objective function: $\left( \textbf{y} - \textbf{y}^{*} \right)^{T} \left( \textbf{y} - \textbf{y}^{*} \right) = \left( y_{1} - y_{1}^{*} \right)^{2} + \left( y_{2} - y_{2}^{*} \right)^{2}$

Reference: $\textbf{y}^{*} = \left[ 1, -1.5 \right]^{T}$

In [ ]:
# Setup and run simulation of simple passthrough system with multiple ES operating in parallel to minimize/maximize two separate objective functions


# define pass-through system
def pass_through_system_01(u):
    # return np.array([[1.0, 0, 0], [0, 1, 0], [0, 0, 1]])@u
    return np.array([[1.0, 0, 0], [0, 1, 0], [0, 0, 1], [1, 1, 1]])@u

# define objhective function
def objective_function_01(y, ystar):
    return np.sum((y - ystar)**2)

# setup time
dT = 0.01 # simulation timestep
time = np.arange(0,30+dT,dT)

# desired system output (reference signal)
ystar = np.zeros((4,len(time)))
ystar[0,:] = 1
ystar[1,:] = -1
ystar[2,:] = 1.5
ystar[3,:] = 3

# initialize pass-through system object
PST01 = PassThroughSystem(time, dT, 3, 4, pass_through_system_01, ystar)

# initialize objective function object
OBJ01 = ObjectiveFunction(time, dT, objective_function_01, ystar)

# initialize 2D ES
ESC01 = ExtremumSeekingSimpleND(time, dT, 3, [1, 1.1, 1.2], [0.2, 0.2, 0.2], [0.2, 0.2, 0.2], "minimize", name="NDES_01")

# pass list of ES algorithm(s) to system object(s)
PST01.set_es_list([ESC01])

# pass list of system(s) to objective function object(s)
OBJ01.set_system_list([PST01])

# intialize simulation
sim = Simulation(time, dT, [PST01], [ESC01], [OBJ01])

# run simuation
sim.run_simulation()
    
print("Simulation Complete")

# plot results
plot_es_results(time, dT, [sim], [PST01], [OBJ01], [ESC01], OBJ01.ystar)
    

In [ ]:
# Plot results in 2D space

fig1, axs = plt.subplots(1,2,figsize=(24,12))

axs[0].plot(OBJ01.ystar[0,:],OBJ01.ystar[1,:],'.',markersize=10,label='Reference value')
axs[0].plot(PST01.y[0,:],PST01.y[1,:],linewidth=2,label='System output')
axs[0].plot(PST01.y[0,0],PST01.y[1,0],'g.',markersize=10,label='Initial system output')
axs[0].plot(PST01.y[0,-1],PST01.y[1,-1],'r.',markersize=10,label='Final system output')
axs[0].set_title("System output and reference value")
axs[0].set_xlabel(r"$y_{1}$")
axs[0].set_ylabel(r"$y_{2}$")
axs[0].legend()


axs[1].plot(ESC01.thetahat[0,:],ESC01.thetahat[1,:],'--',label='ESC setpoint')
axs[1].plot(ESC01.thetahat[0,0],ESC01.thetahat[1,0],'g.',markersize=10,label='Initial ESC setpoint')
axs[1].plot(ESC01.thetahat[0,-1],ESC01.thetahat[1,-1],'r.',markersize=10,label='Final ESC setpoint')
axs[1].plot(ESC01.theta[0,:],ESC01.theta[1,:],label='ESC control value')
axs[1].plot(ESC01.theta[0,0],ESC01.theta[1,0],'g*',markersize=10,label='Initial ESC control')
axs[1].plot(ESC01.theta[0,-1],ESC01.theta[1,-1],'r*',markersize=10,label='Final ESC control')
axs[1].set_title("ESC setpoint and control values")
axs[1].set_xlabel(r"$\hat{\theta}_{c}$ and $\theta_{c}$")
axs[1].set_ylabel(r"$\hat{\theta}_{s}$ and $\theta_{s}$")
axs[1].legend()

plt.show()